# Tensorboard

Example of using:

https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html

https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

Load the data

In [ ]:
from torchvision.datasets import CIFAR10
import torch
import torchvision

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            [0.4914, 0.4822, 0.4465], [0.2470, 0.2434, 0.2615]
        ),
    ]
)

trainset = CIFAR10(root="./CIFAR10", train=True, download=True, transform=transform)
testset = CIFAR10(root="./CIFAR10", train=False, download=True, transform=transform)

(
    trainset_mini,
    _,
) = torch.utils.data.random_split(trainset, [2000, 48000])
testset_mini, _ = torch.utils.data.random_split(testset, [200, 9800])

from torch.utils.data import DataLoader

batch_size = 256

trainloader = DataLoader(
    trainset_mini, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True
)
testloader = DataLoader(
    testset_mini, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True
)

Create model, optimizer and select loss

In [ ]:
from torchvision.models import efficientnet_b0
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = efficientnet_b0()
model.train()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

## Run in colab

In [ ]:
# colab magic command
%load_ext tensorboard
# dir with logs
%tensorboard --logdir runs

Sometimes TB fail to star in colab: # https://stackoverflow.com/questions/64218755/getting-error-403-in-google-colab-with-tensorboard-with-firefox


Start training

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Create log writer
writer = SummaryWriter()
for i, (images, labels) in enumerate(trainloader):
    optimizer.zero_grad()
    outputs = model(images.to(device))
    loss = criterion(outputs, labels.to(device))
    loss.backward()
    # stat collection of any data
    writer.add_scalar("loss", loss.item(), i)
    writer.flush()
writer.close()

Now logs stored on disk in `runs` dir

## API
https://pytorch.org/docs/stable/tensorboard.html

## Run locally

Install:
`pip install tensorboard`


Open terminal and type:
`tensorboard --logdir=runs`

You must see something like this:
`TensorBoard 2.11.2 at http://localhost:6006/ (Press CTRL+C to quit)`

Open link in browser:
`http://localhost:6006/`







## Raw log reading

Official code exists but it did't work
https://www.tensorflow.org/tensorboard/dataframe_api

But we can use this repo: https://github.com/j3soon/tbparse

In [ ]:
!pip install -U tbparse

For single experiment:



In [ ]:
from tbparse import SummaryReader

experiment_name = ""  # like '/Apr07_14-41-50_edc2a15b1764'
log_dir = "runs" + experiment_name

reader = SummaryReader(log_dir)
df = reader.scalars
df.head()

For all logs

In [ ]:
reader = SummaryReader("runs", extra_columns={"dir_name"})
df = reader.scalars
df.head()

Now we can do anything with raw data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(df.loc[:, "value"])
plt.show()

#Weights and Biases

[W&B](https://wandb.ai/site)

https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb